In [ ]:
!pip install transformers -q
!pip install datasets -q
!pip install evaluate -q
!pip install jiwer -q
!pip install sacrebleu -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cud

In [ ]:
import os
cache_dir = "/content/huggingface_cache"
os.makedirs(cache_dir, exist_ok=True)

# Set ALL Hugging Face related cache directories
os.environ["TRANSFORMERS_CACHE"] = os.path.join(cache_dir, "transformers")
os.environ["HF_DATASETS_CACHE"] = os.path.join(cache_dir, "datasets")
os.environ["HF_HOME"] = os.path.join(cache_dir, "hf_home")
os.environ["HF_ASSETS_CACHE"] = os.path.join(cache_dir, "assets")
os.environ["HUGGINGFACE_HUB_CACHE"] = os.path.join(cache_dir, "hub")
os.environ["HF_MODULES_CACHE"] = os.path.join(cache_dir, "modules")

# Create all directories
for dir_path in [os.environ["TRANSFORMERS_CACHE"],
                os.environ["HF_DATASETS_CACHE"],
                os.environ["HF_HOME"],
                os.environ["HF_ASSETS_CACHE"],
                os.environ["HUGGINGFACE_HUB_CACHE"],
                os.environ["HF_MODULES_CACHE"]]:
    os.makedirs(dir_path, exist_ok=True)

# Force datasets to use the new cache
from datasets import config
config.HF_DATASETS_CACHE = os.environ["HF_DATASETS_CACHE"]

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset, Dataset, Audio, DatasetDict
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
import evaluate


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from huggingface_hub import whoami

# Get user info
user_info = whoami()
print(user_info)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


{'type': 'user', 'id': '6573e3d7244aefdfc476f07a', 'name': 'cobrayyxx', 'fullname': 'M Hazim AF', 'email': 'muhammadhazim86@gmail.com', 'emailVerified': True, 'canPay': False, 'periodEnd': None, 'isPro': False, 'avatarUrl': '/avatars/16ef7a7d5351781180384c7f5b4ef392.svg', 'orgs': [{'type': 'org', 'id': '643e51141d0e956d94baaade', 'name': 'kreasof-ai', 'fullname': 'Kreasof AI', 'email': None, 'canPay': False, 'periodEnd': None, 'avatarUrl': 'https://cdn-avatars.huggingface.co/v1/production/uploads/63b6f2e752c02ae8acbaa4d8/r_B-tRS854Qo5EOURHxL4.png', 'roleInOrg': 'write', 'isEnterprise': False}, {'type': 'org', 'id': '67a769f44edabdd17b5c169b', 'name': 'SpeechT', 'fullname': 'SpeechT', 'email': None, 'canPay': False, 'periodEnd': None, 'avatarUrl': 'https://cdn-avatars.huggingface.co/v1/production/uploads/618c42e4f35289d4bcab9cf1/m1k2sALFEwa9B2McXNyOi.png', 'roleInOrg': 'read', 'isEnterprise': False}], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'learning', 'role': 'w

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
model_name = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

In [ ]:
train_dataset = load_dataset("kreasof-ai/be-en-IWSLT2025",split="train+val", streaming=True)

README.md:   0%|          | 0.00/620 [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

In [ ]:
test_dataset = load_dataset("kreasof-ai/be-en-IWSLT2025", split="test", streaming=True)

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

In [ ]:
train_dataset

IterableDataset({
    features: ['audio', 'sentence', 'translation', 'speaker_id'],
    num_shards: 41
})

In [ ]:
test_dataset

IterableDataset({
    features: ['audio', 'sentence', 'translation', 'speaker_id'],
    num_shards: 2
})

# Testing with first 10 data (this is just for checking whether the training code is working or not)

In [ ]:
# Get the first 10 examples
train_dataset = list(train_dataset.take(10))
test_dataset = list(test_dataset.take(10))

In [ ]:
train_dataset = Dataset.from_list(train_dataset)
test_dataset = Dataset.from_list(test_dataset)

# Preprocess Data

In [ ]:
train_dataset = train_dataset.remove_columns(["audio"])  # Remove audio
test_dataset = test_dataset.remove_columns(["audio"])

In [ ]:
df_train = train_dataset.to_pandas()
df_test = test_dataset.to_pandas()

In [ ]:
df_train.shape

(10, 3)

In [ ]:
df_test.shape

(10, 3)

In [ ]:
df_train = df_train.drop_duplicates(subset=["sentence"], keep="first").reset_index(drop=True)
df_test = df_test.drop_duplicates(subset=["sentence"], keep="first").reset_index(drop=True)

In [ ]:
df_train.shape

(10, 3)

In [ ]:
df_test.shape

(10, 3)

In [ ]:
df_train.head()

,sentence,translation,speaker_id
0,Abalumendo babili nabeminina bale ikata amabul...,these men are standing caching tree leaves,261
1,Cifwile uyu ou bali nankwe eulemulanga ifyakuc...,It must be that the person they are with is sh...,381
2,Umulumendo ekele pacipuna elo kumaboko ekete i...,A young man sitted on the chair has a red doll...,241
3,Shili mu musebo,They are on the road.,82
4,Umwaume uwabuuta inkanda aletekenya icimbaya m...,One white man is driving a tractor,351


## remove null

In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sentence     10 non-null     object
 1   translation  10 non-null     object
 2   speaker_id   10 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 372.0+ bytes


In [ ]:
df_train = df_train[~df_train["translation"].isna()]

In [ ]:
df_train.shape

(10, 3)

In [ ]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sentence     10 non-null     object
 1   translation  10 non-null     object
 2   speaker_id   10 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 372.0+ bytes


In [ ]:
cleaned_train = Dataset.from_pandas(df_train)
cleaned_test = Dataset.from_pandas(df_test)

In [ ]:
cleaned_train

Dataset({
    features: ['sentence', 'translation', 'speaker_id'],
    num_rows: 10
})

In [ ]:
cleaned_test

Dataset({
    features: ['sentence', 'translation', 'speaker_id'],
    num_rows: 10
})

In [ ]:
len(cleaned_train["translation"])

10

# Training

In [ ]:
metric_bleu = evaluate.load("sacrebleu")
metric_chrf = evaluate.load("chrf")
metric_wer = evaluate.load("wer")

In [ ]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric_wer.compute(predictions=pred_str, references=label_str)

    bleu = metric_bleu.compute(predictions=pred_str, references=label_str)
    bleu = round(bleu["score"], 2)

    chrf = metric_chrf.compute(predictions=pred_str, references=label_str)
    chrf = round(chrf["score"], 2)

    return {"bleu": bleu, "chrf": chrf, "wer": wer}

In [ ]:
sentences = cleaned_train["sentence"][:5]

# Set the language tokens
tokenizer.src_lang = "bem_Latn"  # Source language: eng in Latin script
tokenizer.tgt_lang = "eng_Latn"  # Target language: hin in Latin script

# Tokenize and set source/target languages
inputs = tokenizer(
    sentences,
    return_tensors="pt",
    padding=True,
    truncation=True
)

# Generate translations
translated = model.generate(**inputs)
translations = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]

# Print translations
for i, translation in enumerate(translations):
    print(f"Input: {sentences[i]}")
    print(f"Translation: {translation}\n")

In [ ]:
def preprocess_function(dataset):
    # Tokenize the source text (English) and target text (Hindi)
    try:
      model_inputs = tokenizer(
          dataset["sentence"],  # Source text column
          text_target=dataset["translation"],  # Target text column
          max_length=128,  # Adjust max length based on your needs
          truncation=True,  # Truncate inputs that exceed the max length
          padding="max_length"  # Pad inputs to the max length
      )
      return model_inputs
    except Exception as e:
      print(e)

In [ ]:
train_tokenized = cleaned_train.map(preprocess_function, batched=True)

In [ ]:
test_tokenized = cleaned_test.map(preprocess_function, batched=True)

In [ ]:
train_tokenized

In [ ]:
test_tokenized

In [ ]:
# adjust the variable below
hf_repo = "kreasof-ai"
output_dir = "nllb-IWSLT2025-bem-en"

In [ ]:
# Fine-tune
# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,               # Directory to save checkpoints
    per_device_train_batch_size=16,      # Adjust batch size based on GPU memory
    gradient_accumulation_steps=1,
    learning_rate=1e-4,                   # Learning rate
    warmup_ratio=0.03,
    num_train_epochs=1,                # Train for 100 epochs
    gradient_checkpointing=True,


    fp16=True,
    hub_model_id=f"{hf_repo}/{output_dir}",  # Change this
    evaluation_strategy="epoch",         # Evaluate at the end of each epoch

    per_device_eval_batch_size=8,
    predict_with_generate=True,          # Generate predictions for validation
    generation_max_length=225,

    save_strategy="epoch",               # Save checkpoints at the end of each epoch
    load_best_model_at_end=True,         # Load the best model at the end of training
    metric_for_best_model="chrf",        # Use BLEU as the evaluation metric
    greater_is_better=True,
    logging_dir="./logs",                # Log directory
    logging_steps=100,
    report_to=["tensorboard"],
    push_to_hub=True,                    # Push model to Hugging Face Hub

)

In [ ]:
# Initialize the trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()

In [ ]:
# Push the final model to Hugging Face Hub
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/cobrayyxx/nllb-IWSLT2025-be-en/commit/0401ae014fee6de57f8c60b78e938a2524fe5fcf', commit_message='End of training', commit_description='', oid='0401ae014fee6de57f8c60b78e938a2524fe5fcf', pr_url=None, repo_url=RepoUrl('https://huggingface.co/cobrayyxx/nllb-IWSLT2025-be-en', endpoint='https://huggingface.co', repo_type='model', repo_id='cobrayyxx/nllb-IWSLT2025-be-en'), pr_revision=None, pr_num=None)